<h1 align="center"><font color="yellow">ActiveLab: Active Learning com re-rotulagem de Dados</font></h1>

<font color="yellow">Data Scientist.: PhD.Eddy Giusepe Chirinos Isidro</font>

Links de estudo:

* [ActiveLab: Active Learning with Data Re-Labeling](https://cleanlab.ai/blog/active-learning/)

* [Effectively Annotate Text Data for Transformers via Active Learning + Re-labeling](https://towardsdatascience.com/effectively-annotate-text-data-for-transformers-via-active-learning-re-labeling-25fe036d79f)

* [Transformers Meet Active Learning: Less Data, Better Performance](https://towardsdatascience.com/transformers-meet-active-learning-less-data-better-performance-4cf931517ff6)


Os `dados rotulados` são essenciais para treinar modelos `supervisionados` de aprendizado de máquina, mas os anotadores de dados geralmente cometem erros. Para atenuar os erros de anotação, pode-se coletar várias anotações por ponto de dados para determinar um rótulo de consenso mais confiável, mas isso pode ser caro! <font colo="orange">Para treinar o melhor modelo de ML com o mínimo de rotulagem de dados</font>, uma pergunta importante é: `quais novos dados devo rotular ou quais dos meus rótulos atuais devem ser verificados novamente?`


Nós temos o `ActiveLab` de código aberto, um novo método de `Active Learning` que acabamos de publicar, que ajuda você a decidir quais dados devem ser rotulados a seguir ou renomeados novamente para melhorar ao máximo seu modelo de ML dentro de um orçamento de anotação limitado. Sob um número total fixo de anotações permitidas, os conjuntos de dados de treinamento construídos com o `ActiveLab` levam a modelos de ML muito melhores em comparação com outros métodos de `Active Learning`. A figura abaixo mostra isso no Dataset de classificação (tabular) Wall Robot :

![](https://cleanlab.ai/_next/static/images/activelab_mainresult-7e139d1a0bbe73ae64c2f9661e6164b0.png.webp)

Aqui começamos com um conjunto de treinamento inicial de `500` exemplos rotulados. Treinamos um `modelo classifier` para várias rodadas, plotando sua *accuracy de teste* após cada uma dessas iterações (modelo `ExtraTrees` à esquerda, `modelo MLP` à direita). Em cada rodada, coletamos anotações adicionais para `100` exemplos, escolhidos desse conjunto de `500` ou de um pool separado de `1500` exemplos inicialmente `não rotulados`. Usamos vários métodos de `Aprendizado Ativo` para decidir quais dados rotular/rerotular a seguir. Ao final de cada rodada, treinamos uma nova cópia de nosso modelo com rótulos de consenso estabelecidos a partir dos dados anotados no momento. 

* `Random` corresponde à seleção aleatória de quais exemplos coletar uma anotação adicional; 

* `Good Random` seleciona aleatoriamente entre os exemplos com o menor número de anotações até aqui (priorizando os dados não rotulados primeiro); 

* `Entropy` e `Uncertainty` são métodos populares de aprendizado ativo baseados em modelo que baseiam a seleção nas previsões probabilísticas de um modelo. 

* O `ActiveLab` também conta com essas previsões para estimar o quão informativo outro rótulo será para cada exemplo, mas nosso método também considera: quantas anotações um exemplo recebeu até agora e sua concordância, e quão confiável cada anotador é em geral em relação ao modelo treinado.


`Resultados similares são válidos para outros modelos`, bem como para datasets de classificação de imagens, conforme mostrado em nosso artigo, que detalha a pesquisa que desenvolveu este método:

[ActiveLab: Active Learning with Re-Labeling by Multiple Annotators](https://arxiv.org/abs/2301.11856)


Nosso artigo compara extensivamente o `ActiveLab` com métodos de aprendizado mais ativos em configurações de vários anotadores e anotadores únicos e contém mais detalhes sobre a figura acima, além de muitos outros que mostram que o `ActiveLab` treina classificadores melhores do que outras estratégias de (re)rotulação de dados.

![](https://cleanlab.ai/_next/static/images/thumb-fcde582980b46048cfc0ee3b9ec6b4fe.png.webp)

# Como utilizo o ActiveLab?

Nosso [notebook](https://github.com/cleanlab/examples/blob/master/active_learning_multiannotator/active_learning.ipynb) tutorial mostra como **uma única linha de código Python** pode produzir pontuações do `ActiveLab` para cada ponto de dados (`independentemente de já estar rotulado ou não`). Pontuações mais baixas indicam pontos de dados para os quais a coleta de um rótulo adicional deve ser mais informativa para nosso modelo atual. De código aberto como parte da biblioteca `cleanlab`, o `ActiveLab` requer apenas as seguintes entradas:


* Rótulos de Dataset armazenados como matriz `multiannotator_labels` cujas linhas correspondem aos exemplos que possuem pelo menos uma anotação, colunas aos rótulos escolhidos por cada anotador (`NA` representando anotações ausentes para exemplos que um determinado anotador não rotulou).

* Probabilidades de classe previstas `pred_probs` de qualquer classificador treinado para esses exemplos, bem como adicionais `pred_probs_unlabeled` para um pool separado de exemplos não rotulados (do mesmo classificador).

```
from cleanlab.multiannotator import get_active_learning_scores

scores_labeled, scores_unlabeled = get_active_learning_scores(multiannotator_labels,
                                                              pred_probs,
                                                              pred_probs_unlabeled
                                                             )
```

As pontuações retornadas por este código são diretamente comparáveis ​​entre os dados rotulados/não-rotulados! Se você quiser coletar `B` anotações adicionais para melhorar mais seu conjunto de dados antes do re-treinamento do modelo, você deve pedir aos anotadores para rotular os exemplos com o `B` pontuações mais baixas (independente de quantas anotações cada exemplo já possui).


Depois que os dados foram rotulados dessa maneira, uma pergunta natural é: `qual é a qualidade geral de cada anotador?` Com outra linha de código, você pode usar o `CROWDLAB` para estimar isso com precisão com base nas anotações coletadas até agora.

```
from cleanlab.multiannotator import get_label_quality_multiannotator

quality_estimates = get_label_quality_multiannotator(multiannotator_labels,
                                                     pred_probs
                                                    )
```

# Como funciona?

O `ActiveLab` ajuda a responder à pergunta principal: `quando é melhor coletar mais um rótulo para um ponto de dados anotado anteriormente em vez de um rótulo para um novo exemplo do pool não rotulado?` 

A resposta depende de quão confiáveis ​​consideramos as anotações atualmente coletadas. Intuitivamente, podemos obter outra opinião para exemplos com apenas uma única anotação de um anotador não confiável ou para exemplos com duas anotações que não concordam. Essa nova rotulagem é particularmente importante quando os danos de treinar um modelo com `dados rotulados incorretamente` não podem ser mitigados simplesmente rotulando novos pontos de dados do pool não rotulado.


Algumas das previsões de ML mais precisas são produzidas por meio da combinação ponderada das saídas de diferentes preditores individuais. `CROWDLAB` é um método para estimar nossa confiança em um rótulo de consenso a partir de dados de vários anotadores, que produz estimativas precisas por meio de um conjunto ponderado de previsão probabilística de qualquer modelo treinado $p_M$ e os rótulos individuais atribuídos por cada anotador $j$. O `ActiveLab` forma uma estimativa de conjunto ponderada semelhante, tratando a seleção de cada anotador como uma decisão probabilística $p_j$ saída por outro preditor:


$$p_{ensemble} = w_M.p_M + w_A.\frac{1}{K} + w_1.p_1 + . . . + w_J.p_J$$


Aqui $p_j=0$ se anotador $j$$ não rotulou um exemplo específico, $w_A$ é o peso do anotador médio e $K$ denota o número de classes em nosso conjunto de dados

Pesos $w_M$, $w_1$, . . . , $w_J$ são atribuídos de acordo com uma estimativa de quão confiável é o modelo versus cada anotador individual.